# Initial Test

* Aluno: Artur Chiaperini Grover
* Exchange: CEX ([CEX.IO](https://cex.io))

In [ ]:
import pickle
import pandas as pd
import numpy as np
import ccxt
import datetime
import requests
import json
import time
import sqlite3
import sqlalchemy
from IPython.display import clear_output

----
## Function definition

In [ ]:
def create_date_list(start, end):
    '''
    create_date_list(start, end):
    returns a list of dates from start to end with format 'YYYYMMDD' and type string.
    
    parameters:
    start -> Starting date with format 'YYYY-MM-DD' (string).
    end   -> Ending date with format 'YYYY-MM-DD' (string).
    '''
    start = datetime.datetime.strptime(start, '%Y-%m-%d')
    end = datetime.datetime.strptime(end, '%Y-%m-%d')
    step = datetime.timedelta(days = 1)
    date_list = list()
    while start <= end:
        date_list.append(str(start.date()).replace('-', ''))
        start += step
        
    return date_list

In [ ]:
def create_url_list(symbol1, symbol2, start, end):
    '''
    create_url_list(symbol1, symbol2, start, end):
    returns a list of urls from start to end date of the symbol1/symbol2 transaction.
    
    parameters:
    symbol1 -> Cryptocurrency acronym (string).
    symbol2 -> Currency acronym (string).
    start   -> Starting date with format 'YYYY-MM-DD' (string).
    end     -> Ending date with format 'YYYY-MM-DD' (string).
    '''
    
    date_list = create_date_list(start, end)
    url_list = list()
    for date in date_list:
        url = 'http://cex.io/api/ohlcv/hd/{}/{}/{}'.format(date, symbol1, symbol2)
        url_list.append(url)
        
    return url_list

In [ ]:
def get_ohlcv(symbol1, symbol2, start, end, data_rate):
    '''
    get_ohlcv(symbol1, symbol2, start, end, data_rate):
    returns a string with all read fetched data from cex.io for the given pair symbol1/symbol2 between
    the starting and ending dates and with data rate (1m, 1h or 1d).
    
    Function has a one second sleep to avoid exploding rate limit of exchange (cex.io).
    Cex rate limit is 600 requests per 10 minutes (1 request per second).
    
    parameters:
    symbol1   -> Cryptocurrency acronym (string).
    symbol2   -> Currency acronym (string).
    start     -> Starting date with format 'YYYY-MM-DD' (string).
    end       -> Ending date with format 'YYYY-MM-DD' (string).
    data_rate -> Data rate, cex.io gives the following options: 'data1m', 'data1h' or 'data1d'.
    '''
    
    url_list = create_url_list(symbol1, symbol2, start, end)
    # time_sleep: guarantees that no more than 1 request are done per second.
    time_sleep = 1.12
    print('Number of urls = {}'.format(len(url_list)))
    print('Time sleep = {}'.format(time_sleep))
    count_input = 0
    data_ohlcv_list = []
    for url in url_list:
        print('Reading {} -----'.format(url))
        request_url = requests.get(url).json()
        if request_url == None:
            print('No data to fetch... -----')
            pass
        else:
            ohlcv = eval(requests.get(url).json()[data_rate])
            data_ohlcv_list.append(ohlcv)
            print('Fetch input data size {}'.format(len(ohlcv)))
            count_input += len(ohlcv)
            print('Finished -----')
        
        time.sleep(time_sleep)
        
    print('Total number of inputs {}'.format(count_input))
    return str([item for sublist in data_ohlcv_list for item in sublist])

----
## Fetching data from Exchange and Saving Pickle File

### Etherium

In [ ]:
# %%time
# cex_eth_usd = get_ohlcv('ETH', 'USD', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_eth_usd = 'ohlcv_eth_usd.pkl'
# pickle.dump(cex_eth_usd, open(file_ohlcv_eth_usd, 'wb'))

In [ ]:
# %%time
# cex_eth_eur = get_ohlcv('ETH', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_eth_eur = 'ohlcv_eth_eur.pkl'
# pickle.dump(cex_eth_eur, open(file_ohlcv_eth_eur, 'wb'))

### Bitcoin

In [ ]:
# %%time
# cex_btc_usd = get_ohlcv('BTC', 'USD', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_btc_usd = 'ohlcv_btc_usd.pkl'
# pickle.dump(cex_btc_usd, open(file_ohlcv_btc_usd, 'wb'))

In [ ]:
# %%time
# cex_btc_eur = get_ohlcv('BTC', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_btc_eur = 'ohlcv_btc_eur.pkl'
# pickle.dump(cex_btc_eur, open(file_ohlcv_btc_eur, 'wb'))

### ZEC - Zcash

In [ ]:
# %%time
# cex_zec_usd = get_ohlcv('ZEC', 'USD', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_zec_usd = 'ohlcv_zec_usd.pkl'
# pickle.dump(cex_zec_usd, open(file_ohlcv_zec_usd, 'wb'))

In [ ]:
# %%time
# cex_zec_eur = get_ohlcv('ZEC', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_zec_eur = 'ohlcv_zec_eur.pkl'
# pickle.dump(cex_zec_eur, open(file_ohlcv_zec_eur, 'wb'))

----
## Renaming and Correcting Datetime

In [ ]:
loaded_ohlcv_eth_usd = pickle.load(open('ohlcv_eth_usd.pkl', "rb"))

In [ ]:
df = pd.read_json(loaded_ohlcv_eth_usd)
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

In [ ]:
format_date = lambda x: datetime.datetime.fromtimestamp(x) 
df['date'] = pd.to_datetime(df['date'].apply(format_date))

In [ ]:
df.set_index('date', inplace = True)

In [ ]:
df

## TESTS

In [1]:

from capturer import Capturer
import pandas as pd
from sqlalchemy import create_engine

In [2]:
zec_usd = Capturer('zec/usd')

In [3]:
zec_usd.get_ohlcv('2017-08-20', '2018-01-02')

Number of urls = 136
Reading http://cex.io/api/ohlcv/hd/20170820/ZEC/USD -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170821/ZEC/USD -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170822/ZEC/USD -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170823/ZEC/USD -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170824/ZEC/USD -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170825/ZEC/USD -----
Uploading to database...
This data is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170826/ZEC/USD -----
Uploading to database...
This data is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170827/ZEC/USD -----
Uploading to database...
This data is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170828/ZEC/USD -----
Uploading t

KeyboardInterrupt: 

In [ ]:
zec_eur = Capturer('zec/eur')

In [ ]:
zec_eur.get_ohlcv('2018-01-01', '2018-01-02')

In [ ]:
conn = create_engine('sqlite:///ohlcv_db.sqlite')

df = pd.read_sql_query('select * from zec_usd;', conn)
df


In [ ]:
df2 = pd.read_sql_query('select * from zec_eur;', conn)
df2

In [ ]:
df.drop_duplicates('date', inplace = True)

In [ ]:
df.set_index('date', inplace = True)

In [ ]:
df

In [6]:
%run -i 'create_db.py'

The database already exists!

The following tables are present: 
btc_eur
start date: 2017-09-01 00:00:00
end date: 2017-09-05 23:40:00

btc_usd
start date: 2017-09-01 00:00:00
end date: 2017-09-05 23:59:00

eth_eur
start date: 2017-09-01 00:19:00
end date: 2017-09-05 23:52:00

eth_usd
start date: 2017-09-01 00:00:00
end date: 2017-09-05 23:59:00

Do you want to overwrite the database? ["y" - yes; "n" - no] y

Overwriting the database...

Please enter starting date (YYYY-MM-DD): 2017-01-22
Please enter ending date (YYYY-MM-DD): 2018-01-22

This might take a while... please be patient.

Number of urls = 366
Reading http://cex.io/api/ohlcv/hd/20170122/ETH/USD -----
Uploading to database...
Fetch input data size 68
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170123/ETH/USD -----
Uploading to database...
Fetch input data size 33
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170124/ETH/USD -----
Uploading to database...
Fetch input data size 38
Finished -----

Reading http://

Reading http://cex.io/api/ohlcv/hd/20170324/ETH/USD -----
Uploading to database...
Fetch input data size 524
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170325/ETH/USD -----
Uploading to database...
Fetch input data size 574
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170326/ETH/USD -----
Uploading to database...
Fetch input data size 349
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170327/ETH/USD -----
Uploading to database...
Fetch input data size 335
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170328/ETH/USD -----
Uploading to database...
Fetch input data size 255
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170329/ETH/USD -----
Uploading to database...
Fetch input data size 371
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170330/ETH/USD -----
Uploading to database...
Fetch input data size 329
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170331/ETH/USD -----
Uploading to database...
Fetch input data size 390
Finished -----



Reading http://cex.io/api/ohlcv/hd/20170529/ETH/USD -----
Uploading to database...
Fetch input data size 1014
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170530/ETH/USD -----
Uploading to database...
Fetch input data size 1257
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170531/ETH/USD -----
Uploading to database...
Fetch input data size 1225
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170601/ETH/USD -----
Uploading to database...
Fetch input data size 1028
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170602/ETH/USD -----
Uploading to database...
Fetch input data size 846
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170603/ETH/USD -----
Uploading to database...
Fetch input data size 793
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170604/ETH/USD -----
Uploading to database...
Fetch input data size 771
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170605/ETH/USD -----
Uploading to database...
Fetch input data size 818
Finished ---

Uploading to database...
Fetch input data size 838
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170803/ETH/USD -----
Uploading to database...
Fetch input data size 754
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170804/ETH/USD -----
Uploading to database...
Fetch input data size 696
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170805/ETH/USD -----
Uploading to database...
Fetch input data size 1083
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170806/ETH/USD -----
Uploading to database...
Fetch input data size 1064
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170807/ETH/USD -----
Uploading to database...
Fetch input data size 1024
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170808/ETH/USD -----
Uploading to database...
Fetch input data size 1101
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170809/ETH/USD -----
Uploading to database...
Fetch input data size 1273
Finished -----

Reading http://cex.io/api/ohlcv/hd/20170810/ETH/USD -

Reading http://cex.io/api/ohlcv/hd/20171007/ETH/USD -----
Uploading to database...
Fetch input data size 650
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171008/ETH/USD -----
Uploading to database...
Fetch input data size 632
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171009/ETH/USD -----
Uploading to database...
Fetch input data size 860
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171010/ETH/USD -----
Uploading to database...
Fetch input data size 712
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171011/ETH/USD -----
Uploading to database...
Fetch input data size 530
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171012/ETH/USD -----
Uploading to database...
Fetch input data size 791
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171013/ETH/USD -----
Uploading to database...
Fetch input data size 1139
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171014/ETH/USD -----
Uploading to database...
Fetch input data size 864
Finished -----


...

This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170311/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170312/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170313/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170314/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170315/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170316/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170317/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170318/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170319/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex

This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170529/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170530/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170531/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170601/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170602/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170603/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170604/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170605/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170606/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex

This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170816/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170817/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170818/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170819/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170820/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170821/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170822/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170823/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex.io/api/ohlcv/hd/20170824/ZEC/EUR -----
This url is EMPTY, moving to the next one...

Reading http://cex

Reading http://cex.io/api/ohlcv/hd/20171021/ZEC/EUR -----
Uploading to database...
Fetch input data size 28
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171022/ZEC/EUR -----
Uploading to database...
Fetch input data size 15
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171023/ZEC/EUR -----
Uploading to database...
Fetch input data size 28
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171024/ZEC/EUR -----
Uploading to database...
Fetch input data size 27
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171025/ZEC/EUR -----
Uploading to database...
Fetch input data size 17
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171026/ZEC/EUR -----
Uploading to database...
Fetch input data size 24
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171027/ZEC/EUR -----
Uploading to database...
Fetch input data size 27
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171028/ZEC/EUR -----
Uploading to database...
Fetch input data size 18
Finished -----

Reading 

Reading http://cex.io/api/ohlcv/hd/20171226/ZEC/EUR -----
Uploading to database...
Fetch input data size 117
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171227/ZEC/EUR -----
Uploading to database...
Fetch input data size 170
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171228/ZEC/EUR -----
Uploading to database...
Fetch input data size 127
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171229/ZEC/EUR -----
Uploading to database...
Fetch input data size 171
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171230/ZEC/EUR -----
Uploading to database...
Fetch input data size 153
Finished -----

Reading http://cex.io/api/ohlcv/hd/20171231/ZEC/EUR -----
Uploading to database...
Fetch input data size 112
Finished -----

Reading http://cex.io/api/ohlcv/hd/20180101/ZEC/EUR -----
Uploading to database...
Fetch input data size 150
Finished -----

Reading http://cex.io/api/ohlcv/hd/20180102/ZEC/EUR -----
Uploading to database...
Fetch input data size 179
Finished -----



In [ ]:
import requests
import json

In [ ]:
data = requests.get('http://cex.io/api/ohlcv/hd/20170922/ZEC/USD').json()

In [ ]:
data

In [ ]:
d = eval(data['data1m'])

In [ ]:
d